In [21]:
filepath = "wta_atp_modified_data.csv"

with open(filepath, "r", encoding='utf-8') as f:
    colnames = f.readline().strip("\n")

colnames = colnames.split(",")
for idx,name in enumerate(colnames):
    print(f"{idx} - {name}")
    pass

0 - "surface"
1 - "player_1_age"
2 - "player_2_age"
3 - "player_1_ht"
4 - "player_2_ht"
5 - "player_1_rank"
6 - "player_2_rank"
7 - "player_1_ace"
8 - "player_2_ace"
9 - "player_1_df"
10 - "player_2_df"
11 - "player_1_1stWon"
12 - "player_2_1stWon"
13 - "player_1_1stIn"
14 - "player_2_1stIn"
15 - "player_1_bpFaced"
16 - "player_2_bpFaced"
17 - "player_1_2ndWon"
18 - "player_2_2ndWon"
19 - "player_1_svpt"
20 - "player_2_svpt"
21 - "player_1_bpSaved"
22 - "player_2_bpSaved"
23 - "player_1_h2h"
24 - "player_2_h2h"
25 - "player_1_win_pct"
26 - "player_2_win_pct"
27 - "player_1_surface_win_pct"
28 - "player_2_surface_win_pct"
29 - "player_1_level_win_pct"
30 - "player_2_level_win_pct"
31 - "winner_binary"


In [28]:
lines = []
surfaces = []
with open(filepath, "r") as f:
    for line in f.readlines():
        line = line.strip("\n").split(",")
        lines.append(line)
        surfaces.append(line[0])

print(type(surfaces))
print(surfaces[:3])
print(set(surfaces))

<class 'list'>
['"surface"', '"Hard"', '"Hard"']
{'"Clay"', '"Grass"', '"surface"', '"Hard"', '"Carpet"'}
